In [2]:
# Dependencies
import json
import requests
from pprint import pprint
from config import eia_api_key 
import pandas as pd
from pandas.io.json import json_normalize



In [3]:
# Building URL & Query
url = "http://api.eia.gov/series"
series_id = "NG.N9010US2.M"
query = f"{url}?api_key={eia_api_key}&series_id={series_id}"

# print(query)

response = requests.get(query).json()
pprint(response)

{'request': {'command': 'series', 'series_id': 'NG.N9010US2.M'},
 'series': [{'copyright': 'None',
             'data': [['201903', 3363602],
                      ['201902', 3043664],
                      ['201901', 3356433],
                      ['201812', 3352785],
                      ['201811', 3233430],
                      ['201810', 3272022],
                      ['201809', 3134786],
                      ['201808', 3180213],
                      ['201807', 3108098],
                      ['201806', 2954943],
                      ['201805', 3081754],
                      ['201804', 2959887],
                      ['201803', 3048056],
                      ['201802', 2723563],
                      ['201801', 2959321],
                      ['201712', 3006062],
                      ['201711', 2884127],
                      ['201710', 2906765],
                      ['201709', 2767345],
                      ['201708', 2780821],
                      ['201707', 2763649]

In [4]:
df = pd.DataFrame(response['series'][0]["data"])
df = df.dropna(how='any')
df.columns = ['Period','Values']
df
    

,Period,Values
0,201903,3363602.0
1,201902,3043664.0
2,201901,3356433.0
3,201812,3352785.0
4,201811,3233430.0
5,201810,3272022.0
6,201809,3134786.0
7,201808,3180213.0
8,201807,3108098.0
9,201806,2954943.0


In [10]:
csv_file = pd.read_csv('tableValueAdded.csv')
csv_file1 = csv_file.dropna(how='any')
csv_file1.head()


,Year,"Agriculture, hunting, forestry, fishing (ISIC A-B)","Mining, Manufacturing, Utilities (ISIC C-E)",Manufacturing (ISIC D),Construction (ISIC F),"Wholesale, retail trade, restaurants and hotels (ISIC G-H)","Transport, storage and communication (ISIC I)",Other Activities (ISIC J-P),Total Value Added
1,1970,35.111423,36.686261,...,3.706791,8.321635,4.395508,11.778382,...
2,1971,33.933005,38.108999,...,3.931784,7.761814,4.412064,11.852334,...
3,1972,32.706472,39.182730,...,3.714151,8.121768,4.818994,11.455885,...
4,1973,33.219650,39.289602,...,3.664466,8.148901,4.752921,10.924461,...
5,1974,33.722328,38.701464,...,3.851050,7.836481,4.675012,11.213664,...


In [7]:
import mysql.connector
from db_key import mysql_user, mysql_password

mydb = mysql.connector.connect(
  host="localhost",
  user=mysql_user,
  passwd=mysql_password)

mycursor = mydb.cursor()

mycursor.execute("CREATE DATABASE IF NOT EXISTS GlobalEnergy_db")



In [8]:
from pandas.io import sql
from sqlalchemy import create_engine
engine = create_engine('mysql+pymysql://' + mysql_user + ':' + mysql_password + '@localhost/GlobalEnergy_db')
with engine.connect() as con, con.begin():
    df.to_sql(con=con, name='Trial_1', if_exists='replace')
    csv_file1.to_sql(con=con, name='csv_1', if_exists='replace')
